In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
def detect_missing_bottel(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    top_left = (130, 1)
    bottom_right = (233, 252)
    segment = gray_image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
    if segment is None or segment.size == 0:
        return False
    black_percentage = np.sum(segment < 150) / segment.size * 100
    return black_percentage < 10

def is_bottle_underfilled(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    roi_top_left = (143, 131)
    roi_bottom_right = (221, 168)
    segment = gray_image[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]
    if segment is None or segment.size == 0:
        return False  
    _, segment_binary = cv2.threshold(segment, 150, 255, cv2.THRESH_BINARY)
    black_percentage = np.count_nonzero(segment_binary == 0) / segment_binary.size * 100
    return black_percentage < 25




def is_bottlelabel_missing(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    top_left = (109, 175)
    bottom_right = (241, 278)

  
    segment = gray_image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

    if segment is None or segment.size == 0:
        return False  

    _, segmented_binary = cv2.threshold(segment, 50, 255, cv2.THRESH_BINARY)

    # Apply morphological closing to fill small gaps in the label
    kernel = np.ones((5, 5), np.uint8)
    segment_closed = cv2.morphologyEx(segmented_binary, cv2.MORPH_CLOSE, kernel)

    
    black_percentage = np.count_nonzero(segment_closed == 0) / segment_closed.size * 100
    return black_percentage > 50


In [13]:
def detect_cap(image, pixels_per_cm, cap_segments):
    bottles_with_cap = []
    # Loop through each segment
    print("------------CAPs --------------")
    for bottle_number, segment_range in enumerate(cap_segments, start=1):
        colors = {'red': 0, 'black': 0, 'white': 0}
   
        # Loop through each segment in the range
        for segment in segment_range:
          
            row = 0
            col = segment
            x_start, y_start = int(col * pixels_per_cm), int(row * pixels_per_cm)
            x_end, y_end = int((col + 1) * pixels_per_cm), int((row + 1) * pixels_per_cm)

            if x_end <= image.shape[1] and y_end <= image.shape[0]:
                # Get the segment image
                segment_image = image[y_start:y_end, x_start:x_end]

                # Convert segment to HSV color space
                hsv_segment = cv2.cvtColor(segment_image, cv2.COLOR_BGR2HSV)

                # Define red color range in HSV
                lower_red = np.array([0, 100, 100])
                upper_red = np.array([10, 255, 255])

                # Threshold the HSV image to get red regions
                red_mask = cv2.inRange(hsv_segment, lower_red, upper_red)

                # Count red, black, and white pixels
                colors['red'] += np.sum(red_mask > 0)

        # Calculate percentages
        total_pixels = (x_end - x_start) * (y_end - y_start)
        percentages = {color: (count / total_pixels) * 100 for color, count in colors.items()}
        # Analyze cap presence and overfilling
       
        has_cap = percentages['red'] > 0
        if has_cap:
            bottles_with_cap.append(bottle_number)
            print(f"Bottle {bottle_number} has a cap.")
        else:
            print(f"Bottle {bottle_number} is missing the cap.")
    return bottles_with_cap

def is_label_not_printed(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
    roi_top_left = (110, 180)
    roi_bottom_right = (240, 280)

    roi = gray_image[roi_top_left[1]:roi_bottom_right[1], roi_top_left[0]:roi_bottom_right[0]]
   
    if roi is None or roi.size == 0:
        return False  

    _, roi_binary = cv2.threshold(roi, 150, 255, cv2.THRESH_BINARY)
    black_percentage = np.count_nonzero(roi_binary == 0) / roi_binary.size * 100
    return black_percentage < 50

def verify_label_alignment(img):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Define the segment coordinates
    segment_top_left = (110, 170)
    segment_bottom_right = (250, 195)

    # Extract the segment for the top part of the label
    segment = gray_image[segment_top_left[1]:segment_bottom_right[1], segment_top_left[0]:segment_bottom_right[0]]

    # Calculate the Sobel gradient in both X and Y directions
    sobel_x = cv2.Sobel(segment, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(segment, cv2.CV_64F, 0, 1, ksize=5)

    # Calculate the magnitude of the Sobel gradient
    sobel_magnitude = np.sqrt(sobel_x ** 2 + sobel_y ** 2)

    # Threshold the Sobel magnitude to get binary image
    _, sobel_binary = cv2.threshold(sobel_magnitude, 50, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(sobel_binary.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables for maximum width and height of bounding boxes
    max_width = 0
    max_height = 0

    # Loop over all contours
    for cnt in contours:
        # Get the bounding box of the contour
        x, y, width, height = cv2.boundingRect(cnt)

        # Update maximum width and height
        max_width = max(max_width, width)
        max_height = max(max_height, height)

    # Extract the segment for the label
    segment_label = gray_image[180:230, 110:250]

    # Calculate the percentage of black pixels in the segment
    black_percentage = 100 * np.mean(segment_label < 50)

    # Check if the label is horizontally aligned
    is_horizontal_aligned = max_width <= 100 or max_height >= 10

    # Check if the black percentage is low
    is_black_percentage_low = black_percentage <= 21

    # Combine both conditions
    is_straight = is_horizontal_aligned and is_black_percentage_low

    return is_straight


def check_bottle_shape(image):
    # Extract the red channel from the input image
    red_channel_image = image[:, :, 2]  # Red channel is at index 2 in Python (0-indexed)

    # Apply contrast adjustment using CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    adjusted_red_channel = clahe.apply(red_channel_image)

    # Extract the segment of interest (ROI) for analysis
    segment_of_interest = adjusted_red_channel[190:280, 100:260]

    # Binarize the segment using a threshold of 200
    _, binary_mask = cv2.threshold(segment_of_interest, 200, 255, cv2.THRESH_BINARY)

    # Find connected components in the binary mask
    num_labels, _, stats, _ = cv2.connectedComponentsWithStats(binary_mask)

    # Initialize variables to store maximum area, width, and height
    max_area, max_width, max_height = 0, 0, 0

    # Loop over all connected components
    for i in range(1, num_labels):
        x, y, width, height, _ = stats[i]
        area = width * height
        if area > max_area:
            max_area = area
            max_width = width
            max_height = height

    # Check if the maximum area, width, and height satisfy the specified conditions
    area_result = (max_area >= 6000) and (max_area <= 8000)
    width_result = (max_width >= 80) and (max_width <= 100)
    height_result = (max_height >= 75) and (max_height <= 90)

    # Combine the results of area, width, and height checks
    overall_result = area_result and width_result and height_result

    return overall_result



def is_overfilled(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    segment_left = 140
    segment_right = 220
    segment_top = 110
    segment_bottom = 140

    if segment_left >= segment_right or segment_top >= segment_bottom:
        print("Invalid segment coordinates.")
        return False

    segment = gray_image[segment_top:segment_bottom, segment_left:segment_right]

  
    if segment is None or segment.size == 0:
        print("Segment is None or empty.")
        return False

  
    _, segment_binary = cv2.threshold(segment, 150, 255, cv2.THRESH_BINARY)

 
    total_pixels = segment_binary.shape[0] * segment_binary.shape[1]
    black_pixel_percentage = 100 * (cv2.countNonZero(segment_binary) / total_pixels)

    result = black_pixel_percentage < 40

    return result


In [15]:
image_path = './Images/image046.jpg'
image = cv2.imread(image_path)
pixels_per_cm = image.shape[1] / 8  
cap_segments = [(1, 2), (4, 5), (7, 8)] 
defects = []

defects.append(detect_cap(image, pixels_per_cm, cap_segments))


is_bottle_missing = detect_missing_bottel(image)
print("------------------")
print("Bottel 2 Missing: ", is_bottle_missing)
print("------------------")
is_underFilled = is_bottle_underfilled(image)
print("Bottel 2 underfilled: ", is_underFilled)
print("------------------")
is_label_missing = is_bottlelabel_missing(image)
print("Bottel 2 Label is Missing: ", is_label_missing)
print("------------------")
is_bottellabel_not_printed = is_label_not_printed(image)
print("Bottel 2 not printing: ", is_bottellabel_not_printed)
print("------------------")
is_label_stright = verify_label_alignment(image)
print("Bottel 2 Label is stright: ", is_label_stright)
print("------------------")

is_bottel_deformed = check_bottle_Shape(image)
print("Bottel 2 out of shape: ", is_bottel_deformed)
print("------------------")
is_bottel_overfilled = is_overfilled(image)
print("Bottel 2 over Filled: ", is_bottel_overfilled)
print("------------------")



------------CAPs --------------
Bottle 1 has a cap.
Bottle 2 is missing the cap.
Bottle 3 has a cap.
------------------
Bottel 2 Missing:  False
------------------
Bottel 2 underfilled:  False
------------------
Bottel 2 Label is Missing:  True
------------------
Bottel 2 not printing:  False
------------------
Bottel 2 Label is stright:  False
------------------
Bottel 2 out of shape:  False
------------------
Bottel 2 over Filled:  True
------------------
